In [1]:
import os
import sys
import warnings
import sklearn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston, load_diabetes
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

from autofeat import AutoFeatRegressor

%matplotlib inline
%load_ext autoreload
%autoreload 2

# ignore all future warnings
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
np.seterr(divide = 'ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
datasets = ["diabetes", "boston"]

# same interface for loading all datasets - adapt the datapath
# to where you've downloaded (and renamed) the datasets
def load_regression_dataset(name, datapath="../datasets/regression/"):
    # load one of the datasets as X and y (and possibly units)
    units = {}
    if name == "boston":
        # sklearn boston housing dataset
        X, y = load_boston(True)

    elif name == "diabetes":
        # sklearn diabetes dataset
        X, y = load_diabetes(True)
    else:
        raise RuntimeError("Unknown dataset %r" % name)
    return np.array(X, dtype=float), np.array(y, dtype=float), units

def test_model(dataset, model, param_grid):
    # load data
    X, y, _ = load_regression_dataset(dataset)
    # split in training and test parts
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)
    
    if model.__class__.__name__ == "SVR":
        sscaler = StandardScaler()
        X_train = sscaler.fit_transform(X_train)
        X_test = sscaler.transform(X_test)
        
    # train model on train split incl cross-validation for parameter selection
    gsmodel = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=5)
    gsmodel.fit(X_train, y_train)
    print("best params:", gsmodel.best_params_)
    print("best score:", gsmodel.best_score_)
    print("MSE on training data:", mean_squared_error(y_train, gsmodel.predict(X_train)))
    print("MSE on test data:", mean_squared_error(y_test, gsmodel.predict(X_test)))
    print("R^2 on training data:", r2_score(y_train, gsmodel.predict(X_train)))
    print("R^2 on test data:", r2_score(y_test, gsmodel.predict(X_test)))
    return gsmodel.best_estimator_

def test_autofeat(dataset, feateng_steps=2):
    # load data
    X, y, units = load_regression_dataset(dataset)
    # split in training and test parts
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)
    # run autofeat
    afreg = AutoFeatRegressor(verbose=1, feateng_steps=feateng_steps, units=units)
    # fit autofeat on less data, otherwise ridge reg model with xval will overfit on new features
    X_train_tr = afreg.fit_transform(X_train, y_train)
    X_test_tr = afreg.transform(X_test)
    print("autofeat new features:", len(afreg.new_feat_cols_))
    print("autofeat MSE on training data:", mean_squared_error(y_train, afreg.predict(X_train_tr)))
    print("autofeat MSE on test data:", mean_squared_error(y_test, afreg.predict(X_test_tr)))
    print("autofeat R^2 on training data:", r2_score(y_train, afreg.predict(X_train_tr)))
    print("autofeat R^2 on test data:", r2_score(y_test, afreg.predict(X_test_tr)))    
    
    # train rreg on transformed train split incl cross-validation for parameter selection
    print("# Ridge Regression")
    rreg = Ridge()
    param_grid = {"alpha": [0.00001, 0.0001, 0.001, 0.01, 0.1, 1., 2.5, 5., 10., 25., 50., 100., 250., 500., 1000., 2500., 5000., 10000.]}
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        gsmodel = GridSearchCV(rreg, param_grid, scoring='neg_mean_squared_error', cv=5)
        gsmodel.fit(X_train_tr, y_train)
    print("best params:", gsmodel.best_params_)
    print("best score:", gsmodel.best_score_)
    print("MSE on training data:", mean_squared_error(y_train, gsmodel.predict(X_train_tr)))
    print("MSE on test data:", mean_squared_error(y_test, gsmodel.predict(X_test_tr)))
    print("R^2 on training data:", r2_score(y_train, gsmodel.predict(X_train_tr)))
    print("R^2 on test data:", r2_score(y_test, gsmodel.predict(X_test_tr)))
        
    print("# Random Forest")
    rforest = RandomForestRegressor(n_estimators=100, random_state=13)
    param_grid = {"min_samples_leaf": [0.0001, 0.001, 0.01, 0.05, 0.1, 0.2]}
    gsmodel = GridSearchCV(rforest, param_grid, scoring='neg_mean_squared_error', cv=5)
    gsmodel.fit(X_train_tr, y_train)
    print("best params:", gsmodel.best_params_)
    print("best score:", gsmodel.best_score_)
    print("MSE on training data:", mean_squared_error(y_train, gsmodel.predict(X_train_tr)))
    print("MSE on test data:", mean_squared_error(y_test, gsmodel.predict(X_test_tr)))
    print("R^2 on training data:", r2_score(y_train, gsmodel.predict(X_train_tr)))
    print("R^2 on test data:", r2_score(y_test, gsmodel.predict(X_test_tr)))
       
    print("# SVR")
    svr = SVR(gamma="scale")
    param_grid = {"C": [1., 10., 25., 50., 100., 250.]}
    sscaler = StandardScaler()
    X_train_tr = sscaler.fit_transform(X_train_tr)
    X_test_tr = sscaler.transform(X_test_tr)
    gsmodel = GridSearchCV(svr, param_grid, scoring='neg_mean_squared_error', cv=5)
    gsmodel.fit(X_train_tr, y_train)
    print("best params:", gsmodel.best_params_)
    print("best score:", gsmodel.best_score_)
    print("MSE on training data:", mean_squared_error(y_train, gsmodel.predict(X_train_tr)))
    print("MSE on test data:", mean_squared_error(y_test, gsmodel.predict(X_test_tr)))
    print("R^2 on training data:", r2_score(y_train, gsmodel.predict(X_train_tr)))
    print("R^2 on test data:", r2_score(y_test, gsmodel.predict(X_test_tr)))

In [3]:
for dsname in datasets:
    print("####", dsname)
    X, y, _ = load_regression_dataset(dsname)
    print(X.shape)

#### diabetes
(442, 10)
#### boston
(506, 13)


In [4]:
for dsname in datasets:
    print("####", dsname)
    rreg = Ridge()
    params = {"alpha": [0.00001, 0.0001, 0.001, 0.01, 0.1, 1., 2.5, 5., 10., 25., 50., 100., 250., 500., 1000., 2500., 5000., 10000., 25000., 50000., 100000.]}
    rreg = test_model(dsname, rreg, params)

#### diabetes
best params: {'alpha': 0.01}
best score: -3043.14487668777
MSE on training data: 2817.5756461735427
MSE on test data: 3119.632550355442
R^2 on training data: 0.541317737800587
R^2 on test data: 0.38300930348673157
#### boston
best params: {'alpha': 1e-05}
best score: -25.427148426837693
MSE on training data: 22.4278718761592
MSE on test data: 20.5580503052922
R^2 on training data: 0.7361592384229154
R^2 on test data: 0.7484031841564716


In [5]:
for dsname in datasets:
    print("####", dsname)
    svr = SVR(gamma="scale")
    params = {"C": [1., 10., 25., 50., 100., 250.]}
    svr = test_model(dsname, svr, params)

#### diabetes
best params: {'C': 10.0}
best score: -3057.7070015538065
MSE on training data: 2577.0841482085507
MSE on test data: 3437.6800004513143
R^2 on training data: 0.5804681274187382
R^2 on test data: 0.32010692168648935
#### boston
best params: {'C': 100.0}
best score: -13.598043246310898
MSE on training data: 3.446934260844429
MSE on test data: 9.636188588435926
R^2 on training data: 0.9594503764998732
R^2 on test data: 0.8820688572255264


In [6]:
for dsname in datasets:
    print("####", dsname)
    rforest = RandomForestRegressor(n_estimators=100, random_state=13)
    params = {"min_samples_leaf": [0.0001, 0.001, 0.01, 0.05, 0.1, 0.2]}
    rforest = test_model(dsname, rforest, params)

#### diabetes
best params: {'min_samples_leaf': 0.05}
best score: -3336.6571277553485
MSE on training data: 2472.319475907154
MSE on test data: 3268.607555103584
R^2 on training data: 0.5975231076301993
R^2 on test data: 0.35354551553768243
#### boston
best params: {'min_samples_leaf': 0.0001}
best score: -10.473222010679008
MSE on training data: 1.41392634158416
MSE on test data: 10.691309401960789
R^2 on training data: 0.9833666161088597
R^2 on test data: 0.8691559091068641


In [7]:
for dsname in datasets:
    print("####", dsname)
    test_autofeat(dsname, feateng_steps=1)

#### diabetes
[AutoFeat] The 1 step feature engineering process could generate up to 70 features.
[AutoFeat] With 353 data points this new feature matrix would use about 0.00 gb of space.
[feateng] Step 1: transformation of original features
[feateng] Generated 45 transformed features from 10 original features - done.
[feateng] Generated altogether 45 new features in 1 steps
[feateng] Removing correlated features, as well as additions at the highest level
[feateng] Generated a total of 36 additional features
[featsel] Scaling data...done.
[featsel] Feature selection run 1/5
[featsel] Feature selection run 2/5
[featsel] Feature selection run 3/5
[featsel] Feature selection run 4/5
[featsel] Feature selection run 5/5
[featsel] 9 features after 5 feature selection runs
[featsel] 9 features after correlation filtering
[featsel] 8 features after noise filtering
[AutoFeat] Computing 3 new features.
[AutoFeat]     3/    3 new features ...done.
[AutoFeat] Final dataframe with 13 feature column

<lambdifygenerated-54>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))


[feateng] Generated altogether 61 new features in 1 steps
[feateng] Removing correlated features, as well as additions at the highest level
[feateng] Generated a total of 22 additional features
[featsel] Scaling data...done.
[featsel] Feature selection run 1/5
[featsel] Feature selection run 2/5
[featsel] Feature selection run 3/5
[featsel] Feature selection run 4/5
[featsel] Feature selection run 5/5
[featsel] 16 features after 5 feature selection runs
[featsel] 15 features after correlation filtering
[featsel] 13 features after noise filtering
[AutoFeat] Computing 6 new features.
[AutoFeat]     6/    6 new features ...done.
[AutoFeat] Final dataframe with 19 feature columns (6 new).
[AutoFeat] Training final regression model.
[AutoFeat] Trained model: largest coefficients:
29.674161094728753
38.009017 * 1/x012
-20.104976 * x004
17.159178 * 1/x007
5.227549 * 1/x002
-3.719025 * 1/x008
2.040892 * x003
0.803623 * x005
-0.521640 * x010
-0.366773 * x012
-0.150635 * x000
0.005017 * x011
-0.

In [8]:
for dsname in datasets:
    print("####", dsname)
    test_autofeat(dsname, feateng_steps=2)

#### diabetes
[AutoFeat] The 2 step feature engineering process could generate up to 2485 features.
[AutoFeat] With 353 data points this new feature matrix would use about 0.00 gb of space.
[feateng] Step 1: transformation of original features
[feateng] Generated 45 transformed features from 10 original features - done.
[feateng] Step 2: first combination of features
[feateng] Generated 1338 feature combinations from 1485 original feature tuples - done.
[feateng] Generated altogether 1495 new features in 2 steps
[feateng] Removing correlated features, as well as additions at the highest level
[feateng] Generated a total of 925 additional features
[featsel] Scaling data...done.
[featsel] Feature selection run 1/5
[featsel] Feature selection run 2/5
[featsel] Feature selection run 3/5
[featsel] Feature selection run 4/5
[featsel] Feature selection run 5/5
[featsel] 25 features after 5 feature selection runs
[featsel] 25 features after correlation filtering
[featsel] 10 features after noi

<lambdifygenerated-1626>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))


[feateng] Generated 2583 feature combinations from 2628 original feature tuples - done.
[feateng] Generated altogether 2645 new features in 2 steps
[feateng] Removing correlated features, as well as additions at the highest level
[feateng] Generated a total of 1190 additional features
[featsel] Scaling data...done.
[featsel] Feature selection run 1/5
[featsel] Feature selection run 2/5
[featsel] Feature selection run 3/5
[featsel] Feature selection run 4/5
[featsel] Feature selection run 5/5
[featsel] 50 features after 5 feature selection runs
[featsel] 39 features after correlation filtering
[featsel] 13 features after noise filtering
[AutoFeat] Computing 13 new features.
[AutoFeat]    13/   13 new features ...done.
[AutoFeat] Final dataframe with 26 feature columns (13 new).
[AutoFeat] Training final regression model.
[AutoFeat] Trained model: largest coefficients:
12.149028244003668
63.577651 * 1/(x007*x012)
22.730324 * 1/(x002*x007)
7.906426 * sqrt(x008)/x012
2.008095 * x005**3/x00

In [9]:
for dsname in datasets:
    print("####", dsname)
    test_autofeat(dsname, feateng_steps=3)

#### diabetes
[AutoFeat] The 3 step feature engineering process could generate up to 60445 features.
[AutoFeat] With 353 data points this new feature matrix would use about 0.09 gb of space.
[feateng] Step 1: transformation of original features
[feateng] Generated 45 transformed features from 10 original features - done.
[feateng] Step 2: first combination of features
[feateng] Generated 5793 feature combinations from 1485 original feature tuples - done.
[feateng] Step 3: transformation of new features


<lambdifygenerated-10409>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))
<lambdifygenerated-10422>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))


<lambdifygenerated-12397>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))
<lambdifygenerated-12410>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))


<lambdifygenerated-13380>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))
<lambdifygenerated-13393>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))


<lambdifygenerated-14350>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))
<lambdifygenerated-14363>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))


...

<lambdifygenerated-86928>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))
<lambdifygenerated-86955>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))
<lambdifygenerated-86968>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))


<lambdifygenerated-86979>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))
<lambdifygenerated-87358>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))
<lambdifygenerated-87378>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))
<lambdifygenerated-87396>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))
<lambdifygenerated-87414>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))
<lambdifygenerated-87432>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))
<lambdifygenerated-87450>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))


<lambdifygenerated-87468>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))
<lambdifygenerated-87478>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))
<lambdifygenerated-87493>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))
<lambdifygenerated-87512>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))
<lambdifygenerated-87529>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))
<lambdifygenerated-87549>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))
<lambdifygenerated-87567>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))
<lambdifygenerated-87585>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))
<lambdifygenerated-87618>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))
<lambdifygenerated-87634>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))
<lambdifygenerated-87646>:2: R

<lambdifygenerated-88135>:2: RuntimeWarning: divide by zero encountered in log
  return (log(t))


[feateng] Generated 41942 transformed features from 10343 original features - done.
[feateng] Generated altogether 54958 new features in 3 steps
[feateng] Removing correlated features, as well as additions at the highest level
[feateng] Generated a total of 24205 additional features
[featsel] Scaling data...done.
[featsel] Feature selection run 1/5
[featsel] Feature selection run 2/5
[featsel] Feature selection run 3/5
[featsel] Feature selection run 4/5
[featsel] Feature selection run 5/5
[featsel] 69 features after 5 feature selection runs
[featsel] 38 features after correlation filtering
[featsel] 27 features after noise filtering
[AutoFeat] Computing 27 new features.
[AutoFeat]    27/   27 new features ...done.
[AutoFeat] Final dataframe with 40 feature columns (27 new).
[AutoFeat] Training final regression model.
[AutoFeat] Trained model: largest coefficients:
39.04342913722523
162.294301 * x005**6/x009**3
23.959305 * 1/(x007 + x012)
14.361764 * 1/(x012*log(x007))
-5.026922 * 1/(s